In [1]:
import os
import pandas as pd
workingDataFile = "/home/chowder/Documents/workingData/polar/hr/polarHRdf.parquet.gzip"
pathOfExport = "/home/chowder/Documents/dataExports/polar/individualSessions/"
dir_list = os.listdir(pathOfExport)

In [2]:
def dfFromFile(fileName):
    rawDf = pd.read_csv(pathOfExport + fileName)
    columnNames = ["sampleDT", "value"]

    startDT = pd.to_datetime(" ".join([rawDf["Date"].loc[0], rawDf["Start time"].loc[0],"-0700"]))
    times = startDT + pd.to_timedelta(rawDf["Sport"].loc[2:]).values
    hrs = rawDf["Date"].loc[2:].apply(lambda x:0 if pd.isna(x) else int(x)).values

    samplesList = [[times[i], hrs[i]] for i in range(len(times))]

    dataDf = pd.DataFrame(data=samplesList, columns=columnNames)
    return dataDf.set_index("sampleDT")

In [3]:
dfToAdd = pd.DataFrame(columns = ["sampleDT", "value"])
dfToAdd = dfToAdd.set_index('sampleDT')

for fn in dir_list:
    dfToAdd = pd.concat([dfToAdd, dfFromFile(fn)])
dfToAdd = dfToAdd[~dfToAdd.index.duplicated(keep='first')]
dfToAdd

/home/chowder/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3448: DtypeWarning: Columns (0,2,3,4,5,6,7,8,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  if await self.run_code(code, result, async_=asy):


,value
sampleDT,
2024-08-23 07:09:21-07:00,66
2024-08-23 07:09:22-07:00,65
2024-08-23 07:09:23-07:00,65
2024-08-23 07:09:24-07:00,64
2024-08-23 07:09:25-07:00,64
...,...
2024-07-30 13:12:42-07:00,118
2024-07-30 13:12:43-07:00,119
2024-07-30 13:12:44-07:00,119


In [4]:
# read working data
fullDF = pd.concat([dfToAdd, pd.read_parquet(workingDataFile)])
fullDF["value"] = fullDF["value"].astype("int")
fullDF = fullDF[~fullDF.index.duplicated(keep='first')]
fullDF

,value
sampleDT,
2024-08-23 07:09:21-07:00,66
2024-08-23 07:09:22-07:00,65
2024-08-23 07:09:23-07:00,65
2024-08-23 07:09:24-07:00,64
2024-08-23 07:09:25-07:00,64
...,...
2024-08-07 22:54:04-07:00,82
2024-08-07 22:54:05-07:00,82
2024-08-07 22:54:06-07:00,83


In [5]:
fullDF.to_parquet(workingDataFile,
              compression='gzip') 